In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn import model_selection

# データファイルを読み込む
input_file = './data/income_data.txt'

# 各変数を定義しておく
Xy = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000

# データファイルをwith構文で読み込む。fはファイルポインタ
with open(input_file, 'r') as f:
    
    # ファイルを1行ずつ読み込む
    for line in f.readlines():
        
        # count_class1tとcount_class2がmax_datapoints件数を超えたらbreakする
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break
        
        # lineの中に?が入ってたら飛ばす
        if '?' in line:
            continue
        
        # ', 'で区切る
        data = line[:-1].split(', ')

        # dataリストの配列の最後のラベルをmax_datapoints読までみ込む。<=50Kだったら。count_class1をカウント。
        if data[-1] == '<=50K' and count_class1 < max_datapoints:
            # リストXyにそのdataをappendする
            Xy.append(data)
            count_class1 += 1

        # dataリストの配列の最後のラベルをmax_datapoints読までみ込む。>50Kだったら。count_class2をカウント
        if data[-1] == '>50K' and count_class2 < max_datapoints:
            Xy.append(data)
            count_class2 += 1

# XyをNumPy配列に変換する
Xy = np.array(Xy)

In [13]:
label_encoder = []

print(Xy)

# 全て0のXy行列を作成する
Xy_encoded = np.empty(Xy.shape)

# income_data.txtの1行目にあるそれぞれのインデックスと値を取り出す
# i:インデックス
# item: 値
print(Xy[0])

# Xy[0]でなんで全ての値を入れるのかが分からない。。。
for i,item in enumerate(Xy[0]):
    print(i, item)
    # 値が数値ならそのまま値をXy_encodedに入れる
    if item.isdigit():
        Xy_encoded[:, i] = Xy[:, i]
    # 値が数値以外（テキストとか）なら
    else:
        # encoderを作成してテキストの値を数値でラベル化する
        encoder = preprocessing.LabelEncoder()        
        Xy_encoded[:, i] = encoder.fit_transform(Xy[:, i])
        label_encoder.append(encoder)

print(Xy_encoded)
# Xに末尾以外を入れる
X = Xy_encoded[:, :-1].astype(int)
# yに末尾のラベルを入れる
y = Xy_encoded[:, -1].astype(int)

# X, yを訓連用に80%、検証用に20%に分ける
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=5)

# 分類器を線形分類器を使う
classifier = LinearSVC(random_state=0)

# 線形分類器で訓練する
classifier.fit(X_train, y_train)

# 線形分類器で予測してテストする
y_test_pred = classifier.predict(X_test)

# F1値を出す
f1 = model_selection.cross_val_score(classifier, X, y, scoring='f1_weighted', cv=3)
print("F1 Score: " + str(round(100*f1.mean(), 2)) + "%")

[['39' 'State-gov' '77516' ... '40' 'United-States' '<=50K']
 ['50' 'Self-emp-not-inc' '83311' ... '13' 'United-States' '<=50K']
 ['38' 'Private' '215646' ... '40' 'United-States' '<=50K']
 ...
 ['58' 'Private' '151910' ... '40' 'United-States' '<=50K']
 ['22' 'Private' '201490' ... '20' 'United-States' '<=50K']
 ['52' 'Self-emp-inc' '287927' ... '40' 'United-States' '>50K']]
['39' 'State-gov' '77516' 'Bachelors' '13' 'Never-married' 'Adm-clerical'
 'Not-in-family' 'White' 'Male' '2174' '0' '40' 'United-States' '<=50K']
0 39
1 State-gov
2 77516
3 Bachelors
4 13
5 Never-married
6 Adm-clerical
7 Not-in-family
8 White
9 Male
10 2174
11 0
12 40
13 United-States
14 <=50K
[[3.90000e+01 5.00000e+00 7.75160e+04 ... 4.00000e+01 3.80000e+01
  0.00000e+00]
 [5.00000e+01 4.00000e+00 8.33110e+04 ... 1.30000e+01 3.80000e+01
  0.00000e+00]
 [3.80000e+01 2.00000e+00 2.15646e+05 ... 4.00000e+01 3.80000e+01
  0.00000e+00]
 ...
 [5.80000e+01 2.00000e+00 1.51910e+05 ... 4.00000e+01 3.80000e+01
  0.00000e+

/Users/tryoutlab/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/tryoutlab/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/tryoutlab/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


F1 Score: 66.82%


/Users/tryoutlab/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [14]:
# 適当な検証データを作成する
input_data = np.array([
    ['37', 'Private', '215646', 'HS-grad', '9', 'Never-married', 
     'Handlers-cleaners', 'Not-in-family', 'White', 'Male', '0', '0', '40',
     'United-States'],
    ['55', 'Private', '287927', 'Doctorate', '16', 'Married-civ-spouse', 
     'Exec-managerial', 'Husband', 'White', 'Female', '15000', '0', '40',
     'United-States']])

# 入力データと同じ行列を初期値0で作成する
input_data_encoded = np.zeros(input_data.shape)
c = 0
for i, item in enumerate(input_data[0]):
    if item.isdigit():
        input_data_encoded[:, i] = input_data[:, i]
    else:
        # label_encoderでラベル値を変換する
        input_data_encoded[:, i] = label_encoder[c].transform(input_data[:, i])
        c += 1

# 分類器で入力データから予測する
predicted_class = classifier.predict(input_data_encoded)
print(label_encoder[-1].inverse_transform(predicted_class))

['<=50K' '>50K']
